# Updated 05-21-2021

<a href="https://colab.research.google.com/github/Mandroide/Traffic_Accident_Detection/blob/main/Anomaly_Detection_Models/Fast_Unsupervised_Anomaly_Detection_Traffic_Videos_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Fast Unsupervised Anomaly Detection in Traffic Videos [(Doshi & Yilmaz, 2020)](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w35/Doshi_Fast_Unsupervised_Anomaly_Detection_in_Traffic_Videos_CVPRW_2020_paper.pdf)

The model implemented in this notebook is the ['Fast Unsupervised Anomaly Detection in Traffic Videos'](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w35/Doshi_Fast_Unsupervised_Anomaly_Detection_in_Traffic_Videos_CVPRW_2020_paper.pdf) model from the University of South Florida [(Doshi & Yilmaz, 2020)](https://openaccess.thecvf.com/content_CVPRW_2020/papers/w35/Doshi_Fast_Unsupervised_Anomaly_Detection_in_Traffic_Videos_CVPRW_2020_paper.pdf). This model won the 2nd place at the [NVIDIA AI City Challenge](https://www.aicitychallenge.org/) 2020 NVIDIA_AICITY 2020 track 4. The original repo and documentation can be found [here](https://github.com/kevaldoshi17/NVIDIA_AICITY)

The expectation of this model is to detect anomalies from traffic videos. The execution with the sample video that is in the repository takes approx. between 10-20 mins.




### Housekeeping Notes:
Before you start working with this notebook, you need decide if you would like to test the code with the [video on the GitHub repository](https://github.com/lopezbec/NVIDIA_AICITY_2020T4/tree/master/sample_video) or with a new set of videos.

If you would like to run a new set of videos (i.e., your own videos) , you would need to clone the repositories to your own Google Drive. Hence, first you would need to "mount" your Google Drive on the Virtual Machine. This will allow you to read and save files from your own Google Drive folder.

To do this, you need to provide the path of your Google Drive for the folder of this assignment and click on the option of “Use_Google_Drive”. Once you provide this information, run this code cell below and follow the instructions. If you don’t click Use_Google_Drive, there is no need to submit a “Working_Directory”

To run with your own set of videos you would need to upload your videos to the directory in your Drive as explained below

In [ ]:
import os
import shutil
import pathlib
from IPython.display import clear_output
#@title Available options
#@markdown Choose if you want to mount your Google drive. _It's recommended to 
#@markdown mount it to be able of accesing the FONDOCyT spreadsheet link with
#@markdown ground truth data._
Use_Google_Drive = True #@param {type:"boolean"}

if Use_Google_Drive:
  import gspread
  from google.colab import drive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials


  auth.authenticate_user()

  #@markdown Default drive directory
 
  Working_Directory = 'My Drive/Colab Notebooks/Anomaly_Detection/Traffic_Accident_Detection/' #@param {type:"string"}
  wd = Working_Directory

  gc = gspread.authorize(GoogleCredentials.get_application_default())
  # Open IoWA DoT Accidents Dataset spreadsheet
  spreadsheet = gc.open_by_key("12paFJB7NinDKwVIDG_abQfCRtXfYMTvsNc-6-9oCDcM")
  drive.mount("/content/gdrive/", force_remount=False)
 
  print("current directory is : " + wd)

ROOT_PATH = pathlib.Path.cwd()
REPO_PATH = ROOT_PATH/"NVIDIA_AICITY"

# **Setting up the Model**


In [ ]:
# Installing the dependencies
!pip3 install natsort
!pip install kneed
!pip install pillow
clear_output()

# Cloning YOLO V4

The original repo can be found at https://github.com/AlexeyAB/darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15287, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15287 (delta 0), reused 1 (delta 0), pack-reused 15283
Receiving objects: 100% (15287/15287), 13.69 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (10377/10377), done.


# Cloning the repository NVIDIA_AICITY


The original repo can be found at https://github.com/kevaldoshi17/NVIDIA_AICITY

In [ ]:
!git clone https://github.com/lopezbec/NVIDIA_AICITY_2020T4.git
%mv NVIDIA_AICITY_2020T4 NVIDIA_AICITY 
%cd {REPO_PATH}
if not os.path.isdir("Data"):
    os.mkdir("Data")

Cloning into 'NVIDIA_AICITY_2020T4'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 138 (delta 63), reused 48 (delta 21), pack-reused 0
Receiving objects: 100% (138/138), 57.02 MiB | 23.77 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/NVIDIA_AICITY


Creating all the necessary directories

In [ ]:
%rm -r "ori_images"
%rm -r "processed_images"
%rm -r "processed_images2" 
%mkdir "ori_images"
%mkdir "processed_images"
%mkdir "processed_images2" 

rm: cannot remove 'ori_images': No such file or directory
rm: cannot remove 'processed_images': No such file or directory
rm: cannot remove 'processed_images2': No such file or directory


# Cloning the tools to measure performance

In [ ]:
%cd {ROOT_PATH}
!git clone "https://github.com/Mandroide/Test_Traffic_Anomaly_Detector"
import sys
sys.path.append(os.fspath(ROOT_PATH/"Test_Traffic_Anomaly_Detector"))
%cd {REPO_PATH}

/content
Cloning into 'Test_Traffic_Anomaly_Detector'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.
/content/NVIDIA_AICITY


# Visualize DOT Iowa Accidents Labels Spreadsheet


In [ ]:
import pandas as pd
#@title Worksheet 2016
worksheet = spreadsheet.worksheet("2016")
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()
df = {}
df[2016] = pd.DataFrame(worksheet.get_all_records())

,User,Accident [Y/N],Year,Folder Name,Video Name,Link,Video Length [sec],Pre collission [sec],Collission [sec],Post collission [sec],"Light conditions [Sunny, Cloudy, Rainy, Showy, Night]","Text on Video [NO, Upper Right corner, Upper Left Corner, Lower Right Corner, Lower Left Corner]",Camera Location (if shown),Day of accident (if shown),Time of collission (if shown),"Type of Accident [Rear ended, With object, T-bone, Other]",Camera Moved [Y/N],Camera Move Time [sec],Other Comments
0,Nelson Abreu,Y,2016,20161127-01 Work Zone Crash,DMTV77--2016_11_27_14_48_07-2016_11_27_15_35_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,2814,186,188,201,"Rainy, Cloudy",Upper Left Corner,South I-35 Hoover Str,11/27/2016,14:51:15,Rear ended,Y,2175,2 cars were in line and one hit other from the...
1,Nelson Abreu,Y,2016,20161129-01 Work Zone Crash,DMTV57--2016_11_29_21_01_46-2016_11_29_22_00_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3500,98,100,107,Night,Upper Left Corner,North I-35/80 Meredith,11/29/2016,21:03:28,With Object and Vehicules,Y,445,The accident involves 3 vehicles but the car l...
2,Nelson Abreu,N,2016,20161212-01 CB fire,CBTV03--2016_12_12_07_56_10-2016_12_12_10_10_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,8032,,,,Sunny,Upper Left Corner,East I-80 24th St,,,,Y,2937,No accident
3,Nelson Abreu,N,2016,20161212-01 CB fire,CBTV04--2016_12_12_07_56_38-2016_12_12_10_10_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,7996,,,,Night,"Top Center, Right Lower Corner",I-29/80 Indian Creek,,,,N,,No accident
4,Nelson Abreu,N,2016,20161212-01 CB fire,CBTV05--2016_12_12_07_56_15-2016_12_12_10_36_1...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,9597,,,,Sunny,"Top Center, Right Lower Corner",I-29/80 Lake Manawa,,,,Y,8532,No accident
5,Nelson Abreu,N,2016,20161212-01 CB fire,CBTV47--2016_12_12_08_28_29-2016_12_12_10_08_2...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,5996,,,,Sunny,Upper Left Corner,I-29/80 East of 24th Street,,,,Y,1187,No accident
6,Nelson Abreu,N,2016,20161212-01 CB fire,CBTV48--2016_12_12_09_52_50-2016_12_12_10_01_3...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,518,,,,Sunny,Upper Left Corner,South Exprs Ramp to I-80WB,,,,N,,No accident
7,Nelson Abreu,N,2016,20161212-01 CB fire,CBTV48--2016_12_12_09_52_56-2016_12_12_09_53_2...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,24,,,,Sunny,Upper Left Corner,South Exprs Ramp to I-80WB,,,,N,,No accident
8,Nelson Abreu,Y,2016,20161126-01 CR Crash,CRTV11--2016_11_26_01_55_08-2016_11_26_03_55_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,7894,,,,NIght,Upper Left Corner,I-380 8th St NE,11/26/2016,,,Y,4918,"Bad Quality Video, looks like there is an acci..."
9,Nelson Abreu,Y,2016,20161126-01 CR Crash,CRTV09--2016_11_26_01_55_12-2016_11_26_03_55_1...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,8963,,,,NIght,Upper Left Corner,I-380 1st Ave SW,11/26/2016,,,Y,5188,"Bad Quality Video, looks like there is an acci..."


In [ ]:
#@title Worksheet 2017
worksheet = spreadsheet.worksheet("2017")
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()
df[2017] = pd.DataFrame(worksheet.get_all_records())

,User,Accident [Y/N],Year,Folder Name,Video Name,Link,Video Length [sec],Pre collission [sec],Collission [sec],Post collission [sec],"Light conditions [Sunny, Cloudy, Rainy, Showy, Night]","Text on Video [NO, Upper Right corner, Upper Left Corner, Lower Right Corner, Lower Left Corner]",Camera Location (if shown),Day of accident (if shown),Time of collission (if shown),"Type of Accident [Rear ended, With object, T-bone, Other]",Camera Moved [Y/N],Camera Move Time [sec],Other Comments
0,Nelson Abreu,N,2017,20171219 DM I-235 Black Sedan Leaving the cras...,DMTV08--2017_12_19_12_56_22-2017_12_19_13_16_2...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,1192,,,,Sunny,Upper Left Corner,East I-235 WB Overpass Center @ 7th St,,,,N,,No accidents
1,Nelson Abreu,N,2017,20171219 DM I-235 Black Sedan Leaving the cras...,DMTV07--2017_12_19_12_56_22-2017_12_19_13_16_2...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,1193,,,,Sunny,Upper Left Corner,East I-235 EB Overpass Center @ 2nd Ave,,,,N,,No accidents
2,Nelson Abreu,N,2017,20171219 AM I-35 Crash,AMTV02--2017_12_19_06_35_07-2017_12_19_06_55_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,1194,,,,Night,Upper Left Corner,Nort I-35 NB @ Heart of IA Tr,,,,N,,No accidents
3,Nelson Abreu,N,2017,20171219 AM I-35 Crash,AMTV01--2017_12_19_06_37_45-2017_12_19_06_57_4...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,1194,,,,Night,Upper Left Corner,South I-35 SB Overpass @ IA 210,,,,N,,
4,Nelson Abreu,Y,2017,20171224 DM I-235 Exit 8A Attenuator Crash,DMTV07--2017_12_24_20_01_28-2017_12_24_20_02_1...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,40,2,4,7,Night,Upper Left Corner,East I-245 E8 Overpass Center @ 2nd Ave,12/24/2017,20:01:33,Direct impact to side road protection,N,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,Nelson Abreu,N,2017,20170203-01 WL US 20 Crash,WLTV17--2017_02_03_13_19_34-2017_02_03_14_00_4...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,2473,,,,Sunny,Upper Left Corner,East Evansdale Water Tower,,,,N,,
340,Nelson Abreu,N,2017,20170203-01 WL US 20 Crash,WLTV10--2017_02_03_13_19_35-2017_02_03_14_02_1...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,2558,,,,Sunny,Upper Left Corner,West I-380 @ Gilbertville Road,,,,N,,
341,Nelson Abreu,N,2017,20170203-01 WL US 20 Crash,WLTV13--2017_02_03_13_19_31-2017_02_03_14_01_1...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,2503,,,,Sunny,Upper Left Corner,East I-380 @ US 20 East,,,,N,,
342,Nelson Abreu,N,2017,20170125-01 SC I-29 crash,SCTV11--2017_01_25_18_44_49-2017_01_25_19_43_0...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3490,,,,Night,Upper Left Corner,South I-29 @ Hamilton,,,,Y,1195,


In [ ]:
#@title Worksheet 2018
worksheet = spreadsheet.worksheet("2018")
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()
df[2018] = pd.DataFrame(worksheet.get_all_records())

,User,Accident [Y/N],Year,Folder Name,Video Name,Link,Video Length [sec],Pre collision [sec],Collision [sec],Post collision [sec],"Light conditions [Sunny, Cloudy, Rainy, Showy, Night]","Text on Video [NO, Upper Right corner, Upper Left Corner, Lower Right Corner, Lower Left Corner]",Camera Location (if shown),Day of accident (if shown),Time of collision (if shown),"Type of Accident [Rear ended, With object, T-bone, Other]",Camera Moved [Y/N],Camera Move Time [sec],Other Comments
0,,Y,2018,20180727 D5 US34 at X23 Car Accident,D5TV02 New London_Small.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,51,25,27,32,Sunny,Upper Right Corner,West US 34 near New London,1/27/2018,16:53:06,T-bone,N,NA,Not much traffic
1,,N,2018,20181010 IC I-380 at Penn Accident,ICTV14--2018_10_10_05_21_20-2018_10_10_12_11_1...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,10830,NA,NA,NA,NA,Upper Right Corner,North I-360 @ Penn St,10/10/2018,NA,NA,NA,NA,"No accident, No video"
2,,N,2018,20181010 IC I-380 at Penn Accident,ICTV17_2018_10_10_09_06_20.mov,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,299,NA,NA,NA,Cloudy,Upper Right Corner,i-360 @ Iowa River,10/10/2018,NA,NA,N,NA,No accident
3,,N,2018,20181010 IC I-380 at Penn Accident,ICTV15--2018_10_10_05_21_20-2018_10_10_12_11_2...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,12600,NA,NA,NA,Night,Upper Right Corner,I-360 @ Swan Lake,10/10/2018,NA,NA,Y,11480,"Accident not on camera, just show traffic jam"
4,,N,2018,20181010 IC I-380 at Penn Accident,ICTV18_2018_10_10_09_11_18.mov,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,296,NA,NA,NA,Cloudy,Upper Right Corner,North I-360 @ Swisher,10/10/2018,NA,NA,N,NA,No accident
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,12,,2016,20180107 DM I-235 Crash,DMTV06--2018_01_07_09_05_58-2018_01_07_09_15_5...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,
593,12,,2016,20180107 DM I-235 Crash,DMTV04--2018_01_07_09_05_58-2018_01_07_09_15_5...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,
594,12,,2016,20180107 DM I-35_80 EB Exit 127 Crash,DMTV20--2018_01_07_16_58_47-2018_01_07_16_59_3...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,
595,13,,2016,20180102 CR I-380 Crash,CRTV07--2018_01_02_15_34_23-2018_01_02_16_09_2...,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,


In [ ]:
#@title Worksheet 2019 (RAs)
worksheet = spreadsheet.worksheet("2019 (RAs)")
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()
df = pd.DataFrame(worksheet.get_all_records())
print(df.keys())
df
#.dropna(axis=0, subset=[["Pre collission [sec]", "Collission [sec]",
#                          "Post collission [sec]"]], thresh=2) 

Index(['Labeler (An/Mar)', 'Accident [Y/N]', 'Year', 'Folder Name',
       'Video Name', 'Link', 'Video Length [sec]', 'Pre collission [sec]',
       'Collission [sec]', 'Post collission [sec]',
       'Light conditions [Sunny, Cloudy, Rainy, Showy, Night]',
       'Text on Video [NO, Upper Right corner, Upper Left Corner, Lower Right Corner, Lower Left Corner]',
       'Camera Location (if shown)', 'Day of accident (if shown)',
       'Time of collission (if shown)',
       'Type of Accident [Rear ended, With object, T-bone, Other]',
       'Camera Moved [Y/N]', 'Camera Move Time [sec]', 'Other Comments'],
      dtype='object')


,Labeler (An/Mar),Accident [Y/N],Year,Folder Name,Video Name,Link,Video Length [sec],Pre collission [sec],Collission [sec],Post collission [sec],"Light conditions [Sunny, Cloudy, Rainy, Showy, Night]","Text on Video [NO, Upper Right corner, Upper Left Corner, Lower Right Corner, Lower Left Corner]",Camera Location (if shown),Day of accident (if shown),Time of collission (if shown),"Type of Accident [Rear ended, With object, T-bone, Other]",Camera Moved [Y/N],Camera Move Time [sec],Other Comments
0,An,Y,2019,20190603 DM I-235 at University Accident,DMTV71-2019-06-03_17h00min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,900,NA,NA,NA,"Sunny, Cloudy",Upper Left Corner,East I-235 EB @E University Av.,06/03/2019,,NA,Y,736,"Accident not on camera, just show 3 cars on th..."
1,An,Y,2019,20190603 DM I-80 Rider corner motorcycle accident,DMTV20-2019-06-03_04h27min30s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,31,NA,NA,NA,Night,Upper Left Corner,East I-35/80 @HWY,06/03/2019,,Motocycle crash,N,,"After the motocycle crash, a car looked to had..."
2,An,Y,2019,20190607 DM I-235 WB at 2nd Ave Crash,DMTV07-2019-06-07_07h59min20s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,120,14,16,22,Sunny,Upper Left Corner,East I-235 @ 2nd Street,06/07/2019,7:59:35,Rear ended,Y,54,The accident involves 2 cars and after the Pos...
3,An,Y,2019,20190607 DM I-80 WB at Merle Hay Crash,DMTV22-2019-06-07_13h36min20s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,60,12,14,43,Sunny,Upper Left Corner,East I-35/80 @Merle Hay,06/07/2019,,T-bone,N,,"The accident involves 2 vehicules, a car and a..."
4,An,Y,2019,20190607 IC I-380 at MM 8 Crash,ICTV17-2019-06-07_08h13min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,240,NA,21,24,Sunny,Upper Left Corner,South I-360 @Iowa Fever,06/07/2019,,NA,Y,222,Accident not so clear on camera. Before the ca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,,,,20190801 QC I-74 at River Drive Hit and Run Ac...,IWZ3113-2019-08-01_20h40min00s000ms,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,
582,,,,20191008 IC I-80 at I-380 Crash,ICTV03-2019-10-08_08h30min00s000ms,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,
583,,,,20191008 IC I-80 at I-380 Crash,ICTV03-2019-10-08_09h12min00s000ms,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,
584,,,,20191008 IC I-80 at I-380 Crash,ICTV03-2019-10-08_09h13min15s000ms,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,,,,,,,,,,,,,


In [ ]:
#@title Worksheet 2020
worksheet = spreadsheet.worksheet("2020")
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()
pd.DataFrame(worksheet.get_all_records())

,User,Accident [Y/N],Year,Folder Name,Video Name,Link,Video Length [sec],Pre collission [sec],Collission [sec],Post collission [sec],"Light conditions [Sunny, Cloudy, Rainy, Showy, Night]","Text on Video [NO, Upper Right corner, Upper Left Corner, Lower Right Corner, Lower Left Corner]",Camera Location (if shown),Day of accident (if shown),Time of collission (if shown),"Type of Accident [Rear ended, With object, T-bone, Other]",Camera Moved [Y/N],Camera Move Time [sec],Other Comments
0,Angel Asencio,N,2020,20200310 DM US 65 Pleasant Hill Debris Crash,DMTV71-2020-03-10_06h00min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3600,,,,Night,No,West I-235 @ University BVD,,,,N,,
1,Angel Asencio,N,2020,20200310 DM US 65 Pleasant Hill Debris Crash,DMTV43-2020-03-10_06h00min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3600,,,,Night,No,I-35 / 80,,,,Y,3249,
2,Angel Asencio,N,2020,20200310 DM US 65 Pleasant Hill Debris Crash,DMTV08-2020-03-10_06h00min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3600,,,,Night,No,I-235 @ 7TH ST,,,,N,,
3,Angel Asencio,N,2020,20200310 DM US 65 Pleasant Hill Debris Crash,DMTV88-2020-03-10_06h00min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3601,,,,Night,No,I-35 / I-80 Rmap @ NE M1Master,,,,N,,
4,Angel Asencio,N,2020,20200310 DM US 65 Pleasant Hill Debris Crash,DMTV46-2020-03-10_06h00min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,3600,,,,Night,No,North 163 University @ Hicksory,,,,N,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Nelson Abreu,N,2020,20200106 IC I-380 at Swisher Crash,ICTV17-2020-01-06_06h35min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,2390,,,,Night,Upper Left Corner,North I-380 @ Iowa River,,,,Y,824,Bad quality video.
137,Nelson Abreu,N,2020,20200106 IC I-380 at Swisher Crash,CTV18-2020-01-06_06h35min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,2401,,,,Night,Upper Left Corner,North I-380 @ Swisher,,,,N,,
138,Nelson Abreu,N,2020,20200107 DM I-235 at Euclid Crash,DMTV01-2020-01-07_10h28min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,600,,,,Sunny,Upper Left Corner,North I-35/80 @ East Mix,,,,N,,
139,Nelson Abreu,N,2020,20200107 DM I-235 at Euclid Crash,DMTV02-2020-01-07_10h28min00s000ms.mp4,https://iastate.app.box.com/s/ltz1a7nn1kxkl04t...,600,,,,Sunny,Upper Left Corner,North I-236 Overpass Center,,,,Y,528,


# Which Video to Run?


 
If you would like to use your own MP4 videos, and you have mounted your Google Drive! (see 1st cell above), select the options "My Videos" and make sure you upload your video(s) to the "Data" folder in the "NVIDIA_AICITY" folder located in your Drive (it might take a few minutes for the new folders to show up in your Drive)

In [ ]:
# define helper functions
import json
import re

from natsort import natsorted
from google.colab import output
from google.colab import files
import stream


def load_sample_video():
    
    id = 1
    video_name ="video_1.mp4"
    path = "sample_video/video_1.mp4"
    data = {"videos": [
                       {"id" : id,
                        "name" : video_name,
                        "path" : path}
                       ]}

    print(data)

    with open('dataset.json','w') as json_file:
          json.dump(data,json_file)
    
    json_file.close


def load_my_video():
    
    data = {}
    data["videos"]=[]

    files_in_data=os.listdir("./Data")
    r= re.compile('.*.mp4')
    files_in_data=list(filter(r.match, files_in_data))

    for i in range(len(files_in_data)):
        id=i+1
        video_name=files_in_data[i]
        path = "Data/"+video_name


        data["videos"].append({
            "id":id,
            "name":video_name,
            "path":path
            })
    print(data)

    with open('dataset.json','w') as json_file:
          json.dump(data,json_file)
    
    json_file.close

# By title
worksheet = spreadsheet.worksheet("2019 (RAs)")

#@markdown **By default, this comes with a small dataset of videos to detect
#@markdown anomalies**

#@markdown **If you wish to use your own dataset folder, check the option and
#@markdown insert a path**
Custom_dataset = True #@param {type:"boolean"}
Custom_drive = Use_Google_Drive and Custom_dataset

if Custom_drive:
  load_my_video()

else:
  load_sample_video()


# Background Modelling

1. Run python extract_frames.py to extract frames from the videos with frequency set as 100.

In [ ]:
!python extract_frames.py 100

2. Run python extract_processed.py to segment the processed images.

In [ ]:
!python extract_processed.py

The next cell change makefile to have **GPU** and **OPENCV** enabled

In [ ]:
%cd ../darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

Verify **CUDA**

In [ ]:
!/usr/local/cuda/bin/nvcc --version

The next cell make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)

In [ ]:
!make
clear_output() 

Download pre-trained Yolov4 weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

3. Run pretrained Yolo v4 model on the processed_images2 folder and save it as result.json. 

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output -dont_show -out result.json < '../NVIDIA_AICITY/images.txt'

In [ ]:
%mv result.json ../NVIDIA_AICITY/result.json

Getting back to NVIDIA_AICITY repository

In [ ]:
%cd ../NVIDIA_AICITY/

# Segmentation Maps

1. Run python3 extract_frames.py to extract frames from the videos with frequency set as 10.

In [ ]:
!python3 extract_frames.py 10

Getting back to the Yolov3 repository

In [ ]:
%cd ../darknet/

2. Run pretrained Yolo v3 model on the original_images folder and save it as part1.json

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output -dont_show -out part1.json < '../NVIDIA_AICITY/ori_images.txt'

Moving part1.json to Mask Folder

In [ ]:
%mv part1.json ../NVIDIA_AICITY/Masks/part1.json

Removing all past masks

In [ ]:
%cd ../NVIDIA_AICITY/Masks
%mkdir Mas
%rm -r *.txt
%cd ..

3. Run Seg_masks.py

In [ ]:
!python3 Seg_masks.py
%cd Masks

4. Run python3 Masks/get_ignore_area.py

In [ ]:
!python3 get_ignore_area.py

In [ ]:
%cd ..

In [ ]:
%rm -r change.npy bounds1.npy bounds2.npy centers1.npy centers2.npy result1.npy result2.npy

In [ ]:
%run "Detector.ipynb"

# Measure performance

In [ ]:
import performance as perf

sheet = "2020"
measure_detection_time_error(65, "1.mp4", spreadsheet, "2020")